In [1]:
from datasets import load_dataset

In [2]:
data = load_dataset('./datasets/ictd')

Reusing dataset ictd (/home/deeplearning/.cache/huggingface/datasets/ictd/plain_text/1.0.0/f80fff22ff9442740a2edcdc26a570ae0720732c4b2ac374528462585b81142a)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
def join_string(string):
    b = []
    for i in range(len(data['train']['table_names'])):
        f = data['train'][i]['table_names'][string]
        #print(f)
        s = ""
        s = s.join(f)
        print(s)
        b.append(s)
    return b

In [4]:
nlqs = join_string("nlq")

tables = join_string("table") 
print(tables[0])

how many cwsn toilets are there in the school xyz?
what is the total purchase done on the ad campaign 'ia-tata-nexon23rd april'?
what is the number of unique clicks for campaign named 'ia-tata-nexon23rd april'?
how many entries are there for campaign name 'ia-tata-nexon23rd april'
which restaurant has hisghest rating in the region xyz
monthly spend based on each region for campaign named 'ia-tata-nexon23rd april'
restaurants in location xyz
percent houses having lpg as cooking fuel in village named xyz
locality with restaurants having highest rating
average clicks for different marketing channel for account named 'pfizer'
max reach for different age group for publisher platform 'instagram'
what is the aggregate score of the school xyz?
percent house havining cars in village named xyz
how many schools in district xyz has functional desktops?
ratio of rural villages to urban villages
how many ad's videos are played 95% of total time?
how many times the ad is shown for stats id '121203210

In [5]:
print(nlqs[0])
print(tables[0])

how many cwsn toilets are there in the school xyz?
wellbeing


In [6]:
dict1=data["train"]["table_names"][0]
dict1['nlq'] = nlqs[0]
dict1['table']=tables[0]
dict1

{'nlq': 'how many cwsn toilets are there in the school xyz?',
 'table': 'wellbeing'}

In [7]:
dict2={}
dict2['query1'] = dict1
dict2

{'query1': {'nlq': 'how many cwsn toilets are there in the school xyz?',
  'table': 'wellbeing'}}

In [8]:
path="/home/deeplearning/data_loading_script/train.json"
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()
tokenizer.train(files=path, vocab_size=30_522, min_frequency=2,
                special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

In [2]:
import os
os.mkdir('./nlqberto')
tokenizer.save_model('nlqberto')

FileExistsError: [Errno 17] File exists: './nlqberto'

Now we have two files that define our new FiliBERTo tokenizer:

- merges.txt — performs the initial mapping of text to tokens
- vocab.json — maps the tokens to token IDs

In [9]:
from transformers import RobertaTokenizer

# initialize the tokenizer using the tokenizer we initialized and saved to file
tokenizer = RobertaTokenizer.from_pretrained('nlqberto', max_len=512)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'RobertaTokenizer'.


In [10]:
tokens = tokenizer('which restaurant has highest rating in the region xyz')

In [11]:
print(tokens)

{'input_ids': [0, 416, 487, 442, 578, 668, 304, 308, 532, 302, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [17]:
tokens.input_ids

[0, 416, 487, 442, 578, 225, 63, 49, 37, 55, 47, 65, 304, 308, 532, 302, 2]

## Now we need three tensors:
- input_ids.
- attention_mask.
- labels.(for now, the label tensor would be the input_ids) 

- now, input_ids needs to be passed through a masked language modelling script. Which will mask around 15% of the tokens withing that tensor.
- while training, model is essentially going to try and guess what those masked tokens are. then optimize the model using the loss between the guesses that the model outputs from the input ids and real values(i.e our labels)

- MLM is one of the two core training approaches. Another one being next sentence prediction.
- Masking is a way to tell sequence-processing layers that certain timesteps in an input are missing, and thus should be skipped when processing the data.
- We will mask around 15% of the tokens.
- So we’re actually inputting an incomplete sentence and asking BERT to complete it for us.
- Example being humans learning fill in the blanks. As humans, we use a mix of general world knowledge, and linguistic understanding to come to that conclusion. For BERT, this guess will come from reading a lot — and learning linguistic patterns incredibly well. 

- Argmax is an operation that finds the argument that gives the maximum value from a target function. 

labels == input_ids

input_ids -> MLM